# Gist Memory Showcase

Welcome to the `gist-memory` showcase! This notebook provides a hands-on introduction to the `gist-memory` library, a toolkit for developing and evaluating text compression strategies for Large Language Model (LLM) contexts.

`gist-memory` helps you manage and compress text data (like dialogue history or large documents) to fit within the limited context windows of LLMs while retaining essential information.

This notebook will walk you through:
1.  **Setup**: Installing `gist-memory` and its dependencies in a Colab environment by downloading and installing from a ZIP archive of the repository.
2.  **Core Functionality**: Defining a custom compression strategy, compressing text, and understanding key objects like `CompressedMemory` and `CompressionTrace`.
3.  **Running Experiments**: Using the library's experiment framework to evaluate compression strategies for ingestion, history retrieval, and response generation tasks.
4.  **Command Line Interface (CLI)**: A brief tour of the `gist-memory` CLI for quick operations without Python scripting.

Let's get started!

## 1. Initial Setup: Install Dependencies

First, we need to set up the environment. This involves downloading a ZIP archive of the `gist-memory` repository, extracting it, installing the package from the local files, and then downloading necessary dependencies and models.

**Note**: If you are running this notebook locally and have already cloned the repository and set up a virtual environment, you might be able to skip some of these steps. However, these cells are designed to work in a fresh Google Colab environment.

### 1.1 Download and Unzip `gist-memory` Repository

In [ ]:
# Download the main branch as a ZIP file
!wget https://github.com/google/gist-memory/archive/refs/heads/main.zip -O gist-memory-main.zip

# Unzip the archive
!unzip -q gist-memory-main.zip
# This creates a directory named gist-memory-main

### 1.2 Install `gist-memory` Package from Local Directory

In [ ]:
# Install the gist-memory package from the unzipped directory
# This method installs the package into site-packages and makes its CLI available.
!pip install ./gist-memory-main

### 1.3 Install Dependencies from Extracted Repo

Install the remaining dependencies listed in `requirements.txt` from the extracted `gist-memory-main` directory. This ensures all features, including those for development and testing shown in this notebook, are available.

In [ ]:
!pip install -r gist-memory-main/requirements.txt

### 1.4 Download spaCy Model

Download the English language model from spaCy, which is used for text processing tasks like sentence segmentation. SpaCy should have been installed as part of the `requirements.txt`.

In [ ]:
!python -m spacy download en_core_web_sm

### 1.5 Set up PYTHONPATH (Usually Not Needed with `pip install`)

When a package is installed using `pip install <local_path>` or `pip install -e <local_path>`, it's typically placed in the Python environment's `site-packages` directory, which is already part of `sys.path`. Therefore, explicitly adding the directory to `PYTHONPATH` for the `gist_memory` library itself is usually not necessary for it to be importable.

The following cell is commented out. You would only need something like this if you were working with a local copy that wasn't installed via `pip` and Python couldn't find the `gist_memory` modules.

In [ ]:
# import sys
# sys.path.append('./gist-memory-main') # Only needed if not installed via pip and Python can't find the package

### 1.6 Download Pre-trained Models for `gist-memory`

Download the default embedding model (for text vectorization) and a small chat model (for response generation experiments) using the `gist-memory` CLI. The CLI should be available after the `pip install ./gist-memory-main` step.

In [ ]:
# Download the default sentence transformer model for embeddings
!gist-memory download-model --model-name all-MiniLM-L6-v2

In [ ]:
# Download a small chat model for demonstration purposes
!gist-memory download-chat-model --model-name distilgpt2

### 1.7 Configure Offline Usage (Optional)

If you have downloaded all necessary models and want to ensure the notebook runs without attempting to access Hugging Face Hub, you can set these environment variables. For this showcase, we'll leave them commented out.

In [ ]:
# For offline use after all models are downloaded, uncomment the following lines:
# import os
# os.environ['HF_HUB_OFFLINE'] = '1'
# os.environ['TRANSFORMERS_OFFLINE'] = '1'

### 1.8 Import Necessary Modules

Import the Python modules from `gist_memory` and other libraries that we'll use throughout this notebook.

In [ ]:
import os
import yaml # For pretty-printing experiment results
from pathlib import Path

# Core gist-memory components for experiments and strategy evaluation
from gist_memory import (
    ExperimentConfig,
    HistoryExperimentConfig,
    ResponseExperimentConfig,
    run_experiment,
    run_history_experiment,
    run_response_experiment,
)
# Components for compression strategies and memory representation
from gist_memory.compression import (
    CompressionStrategy,
    CompressedMemory,
    CompressionTrace,
    StrategyConfig,
    TokenBudget,
    TextBlock
)
from gist_memory.constants import ONBOARDING_DEMO_DIR # Example data directory
from gist_memory.registry import register_compression_strategy # To register custom strategies
from gist_memory.utils import read_text, setup_logging # Utility functions

# Initialize logging for cleaner output
setup_logging()

## 2. Core Functionality: Compression Strategies

At the heart of `gist-memory` is the `CompressionStrategy`. This is an extensible class that defines how text should be compressed. Let's define a simple custom strategy and see how it works.

### 2.1 Define a Custom Compression Strategy

We'll create `TruncateStrategy`, a basic strategy that truncates text to a specified token budget. This demonstrates the fundamental structure of a `CompressionStrategy`:
- It inherits from `CompressionStrategy`.
- It has a unique `id`.
- It implements the `compress` method, which takes a `TextBlock` and a `TokenBudget` and returns a `CompressedMemory` object and a `CompressionTrace` object.

In [ ]:
class TruncateStrategy(CompressionStrategy):
    """A simple strategy that truncates text to the token budget."""
    id = "truncate" # Unique identifier for this strategy

    def __init__(self, config: StrategyConfig):
        super().__init__(config)
        # Initialization specific to this strategy can go here

    def compress(self, text_block: TextBlock, budget: TokenBudget) -> tuple[CompressedMemory, CompressionTrace]:
        """Compresses text by truncating to the budget."""
        input_summary = self.summarize(text_block) # Get summary of input
        
        # Simple truncation based on tokens
        tokens = self.tokenizer.encode(text_block.text)
        truncated_tokens = tokens[:budget.value] # Slice tokens to meet budget
        compressed_text = self.tokenizer.decode(truncated_tokens)
        
        # Create the CompressedMemory object
        compressed_memory = CompressedMemory(
            text=compressed_text,
            interaction_id=text_block.interaction_id, # Preserve interaction context
            source_block_ids=[text_block.id] # Track original source
        )
        output_summary = self.summarize(compressed_memory.to_text_block()) # Get summary of output
        
        # Create the CompressionTrace object for logging and analysis
        trace = CompressionTrace(
            strategy_name=self.id,
            strategy_config=self.config.model_dump(), # Store strategy configuration
            token_budget=budget,
            input_summary=input_summary,
            output_summary=output_summary,
            details={"truncation_details": "Tokens truncated from start"} # Strategy-specific details
        )
        return compressed_memory, trace

### 2.2 Register the Custom Strategy

To make our `TruncateStrategy` available for use by its ID (e.g., in experiments), we register it with the library.

In [ ]:
register_compression_strategy(TruncateStrategy.id, TruncateStrategy)
print(f"Strategy '{TruncateStrategy.id}' registered successfully.")

### 2.3 Load Sample Data

Let's load a sample text file from our extracted `gist-memory-main` directory to use for our compression demonstration. We'll use an excerpt about the moon landing.

In [ ]:
# Define the path to the sample data file within the extracted repository directory.
sample_data_file_path = Path('gist-memory-main/sample_data/moon_landing/01_landing.txt')

# Load the content of the file
if sample_data_file_path.exists():
    sample_text = read_text(sample_data_file_path)
    print(f"Successfully loaded data from {sample_data_file_path}\n")
    print("First 300 characters of the sample data:\n")
    print(sample_text[:300])
else:
    print(f"Error: Sample data file not found at {sample_data_file_path}.")
    print("Please ensure the path is correct relative to your notebook's working directory.")
    # Fallback text if file not found, to allow notebook to continue
    sample_text = ("This is a fallback text because the sample data file was not found. "
                   "Please check the path. This text will be used for demonstration purposes instead.")

### 2.4 Demonstrate Text Compression

Now, we'll use our `TruncateStrategy` to compress the loaded sample text. We need to:
1.  Create a `StrategyConfig`, which can specify things like the tokenizer to use.
2.  Instantiate our `TruncateStrategy` with this config.
3.  Create a `TextBlock` from our sample text.
4.  Define a `TokenBudget`.
5.  Call the `compress` method.

In [ ]:
# 1. Create a StrategyConfig for TruncateStrategy.
# We'll use 'all-MiniLM-L6-v2' as the tokenizer, which should be available from the download step.
truncate_strategy_config = StrategyConfig(name="truncate_demo_config", tokenizer_name="all-MiniLM-L6-v2")

# 2. Create an instance of the TruncateStrategy
truncate_compressor = TruncateStrategy(config=truncate_strategy_config)

# 3. Define a TextBlock for compression
# TextBlock wraps the input text and associated metadata.
text_to_compress_block = TextBlock(id="moon_landing_excerpt", text=sample_text, interaction_id="demo_interaction_01")

# 4. Define a token budget (e.g., compress to a maximum of 75 tokens)
compression_budget = TokenBudget(value=75, type="max_tokens")

# 5. Compress the text
compressed_memory_output, compression_trace_output = truncate_compressor.compress(text_to_compress_block, compression_budget)

print(f"Compression complete using strategy '{truncate_compressor.id}'.")

### 2.5 Understanding `CompressedMemory` and `CompressionTrace`

The `compress` method returned two important objects:

#### `CompressedMemory`

The `CompressedMemory` object encapsulates the result of the compression. Its most important attribute is `text`, which contains the compressed version of the input. It also stores metadata linking it to the original text and the interaction it belongs to.

In [ ]:
print("--- Compressed Text Output ---:\n")
print(compressed_memory_output.text)
print(f"\nOriginal Text Length (tokens): {compression_trace_output.input_summary.num_tokens}")
print(f"Compressed Text Length (tokens): {compression_trace_output.output_summary.num_tokens}")

#### `CompressionTrace`

The `CompressionTrace` object logs detailed metadata about the compression process. This is invaluable for debugging, analysis, and understanding the behavior of a strategy. It includes:
- The strategy name and its configuration.
- The token budget applied.
- Summaries of the input and output text (character counts, word counts, sentence counts, token counts).
- Any custom details logged by the strategy.

In [ ]:
print("--- Compression Trace Information ---:\n")
print(f"Strategy Name: {compression_trace_output.strategy_name}")
print(f"Strategy Configuration: {compression_trace_output.strategy_config}")
print(f"Token Budget: {compression_trace_output.token_budget.value} {compression_trace_output.token_budget.type}")
print(f"Input Summary: Chars={compression_trace_output.input_summary.num_chars}, Words={compression_trace_output.input_summary.num_words}, Sents={compression_trace_output.input_summary.num_sents}, Tokens={compression_trace_output.input_summary.num_tokens}")
print(f"Output Summary: Chars={compression_trace_output.output_summary.num_chars}, Words={compression_trace_output.output_summary.num_words}, Sents={compression_trace_output.output_summary.num_sents}, Tokens={compression_trace_output.output_summary.num_tokens}")
print(f"Strategy-Specific Details: {compression_trace_output.details}")

## 3. Running Experiments

`gist-memory` provides a framework for systematically evaluating compression strategies. There are three main types of experiments, configured using specific data classes:

### Experiment Configuration Objects:
-   **`ExperimentConfig`**: Used for *ingestion experiments*. These evaluate how well a strategy compresses a single, potentially large, text document. Key metrics often include compression ratio, information preservation (if measurable via summarization or other techniques), and processing time.
-   **`HistoryExperimentConfig`**: Used for *history retrieval experiments*. These assess a strategy's ability to compress dialogue history while retaining information that is useful for retrieving relevant past turns or facts. Metrics typically involve retrieval scores like Mean Reciprocal Rank (MRR) or Recall@k against ground-truth relevant items.
-   **`ResponseExperimentConfig`**: Used for *response generation experiments*. These evaluate the quality of LLM-generated responses when the compressed dialogue history (produced by the strategy) is used as context. Evaluation can involve automated metrics (e.g., ROUGE, BLEU for summarization/translation tasks) or human evaluation.

### 3.1 Run Ingestion Experiment

This experiment type evaluates how well a strategy compresses a single document. We'll use the `TruncateStrategy` and our moon landing text from the `gist-memory-main` directory.

In [ ]:
# Path to the data for the ingestion experiment (reusing the sample data path from the extracted repo)
ingestion_data_path = sample_data_file_path # Path('gist-memory-main/sample_data/moon_landing/01_landing.txt')

if ingestion_data_path.exists():
    # Create an ExperimentConfig for ingestion
    # We use our registered 'truncate' strategy and its configuration.
    ingestion_exp_config = ExperimentConfig(
        id="ingestion_demo_experiment",
        experiment_type="ingestion",
        strategy_id=TruncateStrategy.id, # Using the ID of our registered strategy
        strategy_config=truncate_strategy_config, # Using the config created earlier
        dataset_path=str(ingestion_data_path), # Path to the text file
        token_budgets=[50, 100], # Test with different token budgets
        output_dir="./ingestion_experiment_output" # Directory to save results
    )

    # Run the ingestion experiment
    print(f"Running ingestion experiment: {ingestion_exp_config.id}")
    ingestion_metrics_results = run_experiment(ingestion_exp_config)

    # Print the metrics in a readable YAML format
    print("\n--- Ingestion Experiment Metrics ---:\n")
    print(yaml.safe_dump(ingestion_metrics_results, indent=2, sort_keys=False))
else:
    print(f"Error: Ingestion data file not found at {ingestion_data_path}. Skipping ingestion experiment.")

The output above typically includes metrics like compression ratio, number of tokens before and after compression for each budget, and processing time.

### 3.2 Run History Experiment

This experiment evaluates a strategy's ability to compress dialogue history for effective retrieval. It uses a dataset of dialogues (from `gist-memory-main`) where specific past turns are marked as relevant to current turns.

**Note**: The `truncate` strategy is very naive for history retrieval. More sophisticated strategies (like those using summarization or embedding similarity) would likely perform better here. This is for demonstration of the experiment mechanics.

In [ ]:
# Define the path to the history dialogues data from the extracted repo
history_dialogues_data_path = Path('gist-memory-main/tests/data/history_dialogues.yaml')

if history_dialogues_data_path.exists():
    # Create a HistoryExperimentConfig
    # We'll use the 'truncate' strategy and its config again.
    history_exp_config = HistoryExperimentConfig(
        id="history_demo_experiment",
        experiment_type="history_retrieval",
        strategy_id=TruncateStrategy.id,
        strategy_config=truncate_strategy_config,
        dataset_path=str(history_dialogues_data_path),
        # param_grid allows testing different strategy parameters or budgets
        param_grid={"token_budget": [TokenBudget(value=100, type="max_tokens"), TokenBudget(value=150, type="max_tokens")]},
        output_dir="./history_experiment_output"
    )

    # Run the history experiment
    print(f"Running history experiment: {history_exp_config.id}")
    history_experiment_results = run_history_experiment(history_exp_config)

    # Print the results
    print("\n--- History Experiment Results ---:\n")
    print(yaml.safe_dump(history_experiment_results, indent=2, sort_keys=False))
else:
    print(f"Error: History dialogues data not found at {history_dialogues_data_path}. Skipping history experiment.")

The results from a history experiment usually include retrieval metrics like MRR (Mean Reciprocal Rank) and Recall@k, indicating how well the compressed history helped in identifying relevant prior turns.

### 3.3 Run Response Experiment

A response experiment assesses how compressed memory affects the quality of responses from an LLM. It uses a dataset of dialogues (from `gist-memory-main`) where the task is to generate a response based on the history.

**Note**: This experiment can take longer to run as it involves calls to an LLM (even a small local one like `distilgpt2`). The `truncate` strategy's impact on response quality might be detrimental if too much context is lost.

In [ ]:
# Define the path to the response dialogues data from the extracted repo
response_dialogues_data_path = Path('gist-memory-main/tests/data/response_dialogues.yaml')

if response_dialogues_data_path.exists():
    # Create a ResponseExperimentConfig
    # We use 'distilgpt2' as the chat model, downloaded during setup.
    response_exp_config = ResponseExperimentConfig(
        id="response_demo_experiment",
        experiment_type="response_generation",
        strategy_id=TruncateStrategy.id, 
        strategy_config=truncate_strategy_config,
        dataset_path=str(response_dialogues_data_path),
        param_grid={"token_budget": [TokenBudget(value=100, type="max_tokens")]},
        chat_model_name="distilgpt2", # LLM for generating responses
        output_dir="./response_experiment_output"
    )

    # Run the response experiment
    print(f"Running response experiment: {response_exp_config.id}")
    # This can take a few minutes depending on the dataset size and model
    response_experiment_results = run_response_experiment(response_exp_config)

    # Print the results
    print("\n--- Response Experiment Results ---:\n")
    print(yaml.safe_dump(response_experiment_results, indent=2, sort_keys=False))
else:
    print(f"Error: Response dialogues data not found at {response_dialogues_data_path}. Skipping response experiment.")

Response experiment results often include metrics like ROUGE or BLEU (if reference responses are available and the task is suitable), or qualitative examples of generated responses. The output here will show file paths where detailed per-instance results and generated texts are stored.

## 4. Command Line Interface (CLI) Showcase

`gist-memory` also features a versatile Command Line Interface (CLI) for performing common operations without needing to write Python scripts. This is handy for quick tests, batch processing, model downloads, and direct interaction with compression strategies. The CLI became available after we installed `gist-memory` using `pip`.

### 4.1 Basic Help Command

To view all available CLI commands and their general options, use the `--help` flag.

In [ ]:
!gist-memory --help

### 4.2 Compression Command (`compress`)

The `compress` command lets you apply a compression strategy to text input directly or from a file. 

**Important Note**: The CLI uses strategies that are built into `gist-memory` or registered via its plugin system. The custom `TruncateStrategy` we defined and registered within this Python notebook session is *not* automatically available to the standalone CLI. For CLI examples, we'll use `truncate` if it's a built-in alias for a simple truncation strategy, or another standard built-in strategy like `gist` (which aims to create a gist of the text).

In [ ]:
# Example: Compress a short text string using the 'truncate' strategy (if available as a built-in/plugin)
# If 'truncate' isn't a known CLI strategy, this might default to another or error.
# Let's assume a basic 'truncate' or 'passthrough' might be available for such a small budget.
!gist-memory compress --strategy truncate --text "This is a fairly long sentence that we want to compress using the command line interface to a small number of tokens." --budget 20

# Example: Compress a text file using the 'gist' strategy
# The path needs to point into the extracted 'gist-memory-main' directory.
cli_sample_file = 'gist-memory-main/sample_data/moon_landing/01_landing.txt'
if Path(cli_sample_file).exists():
  !gist-memory compress --strategy gist --file {cli_sample_file} --budget 100
else:
  print(f"CLI sample file not found: {cli_sample_file}")

Common options for `gist-memory compress`:
-   `--strategy <name>`: ID of the compression strategy (e.g., `gist`, `truncate`).
-   `--text "<string>"`: The text string to compress.
-   `--file <path>`: Path to a text file for compression.
-   `--budget <int>`: The target token budget.
-   `--tokenizer <name>`: (Optional) Specify a tokenizer if the strategy shouldn't use its default.
-   `--output <path>`: (Optional) Path to save the compressed output (e.g., as JSON).

### 4.3 Talk Command (`talk`)

The `talk` command enables interactive chat with an LLM that uses a specified compression strategy to manage the conversation history. This is great for qualitatively evaluating how different strategies affect conversation flow and context retention.

Running a fully interactive `talk` session is not feasible within a static notebook cell. Instead, we'll show how to get help for the command and a conceptual example of its use.

In [ ]:
!gist-memory talk --help

**Conceptual Usage of `talk`:**

You would typically run these in your terminal:

1.  **Talk with a strategy applied to the ongoing conversation history:**
    ```bash
    gist-memory talk --strategy gist --chat-model distilgpt2 --budget 150
    ```
    This starts an interactive session where `distilgpt2` is the LLM, and the `gist` strategy compresses the conversation history to about 150 tokens before each LLM call.

2.  **Talk using a pre-compressed memory file:**
    First, compress a document (using the extracted repo path):
    ```bash
    gist-memory compress --strategy gist --file ./gist-memory-main/sample_data/moon_landing/01_landing.txt --budget 200 --output moon_memory.json
    ```
    Then, start a conversation using this memory as initial context:
    ```bash
    gist-memory talk --memory moon_memory.json --chat-model distilgpt2 --message "What were the main objectives?"
    ```

For a non-interactive check within the notebook, you can try sending a single message. The behavior might vary:


In [ ]:
# This attempts a non-interactive query using the 'truncate' strategy with a specified chat model.
# The CLI might provide a single response or indicate it's for interactive use.
!gist-memory talk --strategy truncate --chat-model distilgpt2 --message "What is the capital of France?" --budget 50

### 4.4 Other Useful CLI Commands

The `gist-memory` CLI offers other utilities. Here are a few, with examples of how to get their help messages:

-   `download-model`, `download-chat-model`: We used these in the setup to fetch models (e.g., `all-MiniLM-L6-v2`, `distilgpt2`).
-   `stats`: Provides statistics about datasets or memory files.
-   `validate`: Validates configuration files or memory formats against the library's schemas.

In [ ]:
!gist-memory stats --help
!echo "\n---------------------------------------\n" # Visual separator
!gist-memory validate --help

## 5. Conclusion and Next Steps

This notebook has provided a tour of `gist-memory`, covering:
-   **Installation and Setup**: Getting your environment ready by downloading a ZIP of the repository, installing the package locally, and installing dependencies.
-   **Core Concepts**: Defining custom `CompressionStrategy` classes, and understanding `CompressedMemory` and `CompressionTrace`.
-   **Experiment Framework**: Running ingestion, history retrieval, and response generation experiments to evaluate strategies, using data from the extracted repository.
-   **CLI Usage**: Interacting with `gist-memory` via the command line for compression and other utilities.

We hope this showcase helps you get started with `gist-memory` for your own LLM context management tasks!

### Further Resources:
-   **GitHub Repository**: [https://github.com/google/gist-memory](https://github.com/google/gist-memory) (The `gist-memory-main` directory we created is an extraction of a ZIP from this.)
-   **Documentation**: Check the `docs/` directory in the `gist-memory-main` folder for more in-depth information.
-   **Examples**: Explore other examples in the `gist-memory-main/examples/` directory.

Feel free to open issues on GitHub for questions, bug reports, or feature requests.